In [12]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras import preprocessing

In [13]:
INPUT_SIZE = (1, 28, 28)

input = tf.placeholder(tf.float32, shape = INPUT_SIZE)
dropout = tf.keras.layers.Dropout(rate = 0.2)(input)
conv = tf.keras.layers.Conv1D(
    filters = 10,
    kernel_size = 3,
    padding = 'same',
    activation = tf.nn.relu)(dropout)
max_pool = tf.keras.layers.MaxPool1D(
    pool_size = 3,
    padding = 'same')(conv)
flatten = tf.keras.layers.Flatten()(max_pool)
hidden = tf.keras.layers.Dense(units = 50, activation = tf.nn.relu)(flatten)
output = tf.keras.layers.Dense(units = 10, activation = tf.nn.softmax)(hidden)

In [14]:
samples = ['너 오늘 이뻐 보인다',
          '나는 오늘 기분이 더러워',
          '끝내주는데, 좋은 일이 있나봐',
          '나 좋은 일이 생겼어',
          '아 오늘 진짜 짜증나',
          '환상적인데, 정말 좋은거 같아']

label = [[1], [0], [1], [1], [0], [1]]

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)
word_index = tokenizer.word_index

In [15]:
dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()
with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break

(array([4, 1, 5, 6]), array([1]))
(array([7, 1, 8, 9]), array([0]))
(array([10,  2,  3, 11]), array([1]))
(array([12,  2,  3, 13]), array([1]))
(array([14,  1, 15, 16]), array([0]))
(array([17, 18, 19, 20]), array([1]))


In [16]:
dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
EPOCH = 2
dataset = dataset.repeat(EPOCH)
BATCH_SIZE = 2
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.shuffle(len(sequences))

iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break

(array([[4, 1, 5, 6],
       [7, 1, 8, 9]]), array([[1],
       [0]]))
(array([[14,  1, 15, 16],
       [17, 18, 19, 20]]), array([[0],
       [1]]))
(array([[14,  1, 15, 16],
       [17, 18, 19, 20]]), array([[0],
       [1]]))
(array([[10,  2,  3, 11],
       [12,  2,  3, 13]]), array([[1],
       [1]]))
(array([[10,  2,  3, 11],
       [12,  2,  3, 13]]), array([[1],
       [1]]))
(array([[4, 1, 5, 6],
       [7, 1, 8, 9]]), array([[1],
       [0]]))


In [17]:
def mapping_fn(X, Y=None):
    input = {'x': X}
    label = Y
    return input, label

BATCH_SIZE = 2
EPOCH = 2

dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.map(mapping_fn)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.shuffle(len(sequences))
dataset = dataset.repeat(EPOCH)

iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break

({'x': array([[10,  2,  3, 11],
       [12,  2,  3, 13]])}, array([[1],
       [1]]))
({'x': array([[4, 1, 5, 6],
       [7, 1, 8, 9]])}, array([[1],
       [0]]))
({'x': array([[14,  1, 15, 16],
       [17, 18, 19, 20]])}, array([[0],
       [1]]))
({'x': array([[4, 1, 5, 6],
       [7, 1, 8, 9]])}, array([[1],
       [0]]))
({'x': array([[14,  1, 15, 16],
       [17, 18, 19, 20]])}, array([[0],
       [1]]))
({'x': array([[10,  2,  3, 11],
       [12,  2,  3, 13]])}, array([[1],
       [1]]))


In [18]:
import tensorflow as tf
from tensorflow.keras import preprocessing

samples = ['너 오늘 이뻐 보인다',
          '나는 오늘 기분이 더러워',
          '끝내주는데, 좋은 일이 있나봐',
          '나 좋은 일이 생겼어',
          '아 오늘 진짜 짜증나',
          '환상적인데, 정말 좋은거 같아']

label = [[1], [0], [1], [1], [0], [1]]

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)
word_index = tokenizer.word_index

EPOCH = 100

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
    dataset = dataset.repeat(EPOCH)
    dataset = dataset.batch(1)
    dataset = dataset.shuffle(len(sequences))
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

VOCAB_SIZE = len(word_index) + 1
EMB_SIZE = 128

def model_fn(features, labels, mode):
    
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT
    
    embed_input = tf.keras.layers.Embedding(VOCAB_SIZE, EMB_SIZE)(features)
    embed_input = tf.reduce_mean(embed_input, axis = 1)
    
    hidden_layer = tf.keras.layers.Dense(128, activation = tf.nn.relu)(embed_input)
    output_layer = tf.keras.layers.Dense(1)(hidden_layer)
    output = tf.nn.sigmoid(output_layer)
    
    loss = tf.losses.mean_squared_error(output, labels)
    
    if TRAIN:
        global_step = tf.train.get_global_step()
        train_op = tf.train.AdamOptimizer(1e-3).minimize(loss, global_step)
        
        return tf.estimator.EstimatorSpec(
            mode = mode,
            train_op = train_op,
            loss = loss)
    
DATA_OUT_PATH = '/Users/ehrme/data_out/'
    
import os
    
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)
        
estimator = tf.estimator.Estimator(model_fn = model_fn, model_dir = DATA_OUT_PATH + 'checkpoint/dnn')
estimator.train(train_input_fn)
    

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/Users/ehrme/data_out/checkpoint/dnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002B340E38DC8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.